# @chain 데코레이터를 사용하여 Runnable 객체 생성

`@chain` 데코레이터를 추가하여 임의의 함수를 체인으로 변환할 수 있습니다.

이는 함수를 [`RunnableLambda`](./functions)로 래핑하는 것과 기능적으로 동일합니다.

실제 동작을 살펴보겠습니다!


In [ ]:
%pip install -qU langchain langchain-openai

In [7]:
import os

# LangSmith 추적
os.environ["LANGCHAIN_PROJECT"] = "Chain Decorator"

In [8]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import chain
from langchain_openai import ChatOpenAI

`ChatPromptTemplate` 클래스를 사용하여 두 개의 프롬프트 템플릿을 정의합니다.

- `prompt1` 은 주어진 주제에 대한 짧은 설명을, `prompt2` 는 영어로 번역해 달라는 요청 프롬프트 입니다.


In [15]:
# 프롬프트 템플릿을 정의합니다.
prompt1 = ChatPromptTemplate.from_template("{topic} 에 대해 짧게 한글로 설명해주세요.")
prompt2 = ChatPromptTemplate.from_template(
    "{sentence} 를 emoji를 활용한 인스타그램 게시글로 만들어주세요."
)

`custom_chain` 함수는 입력 텍스트를 기반으로 사용자 정의 체인을 실행합니다.

- `@chain` 데코레이터로 사용자 정의 함수를 데코레이팅 하며, 데코레이팅을 통해 함수를 `Runnable` 한 객체로 만듭니다.


In [16]:
@chain
def custom_chain(text):
    # 첫 번째 프롬프트, ChatOpenAI, 문자열 출력 파서를 연결하여 체인을 생성합니다.
    chain1 = prompt1 | ChatOpenAI() | StrOutputParser()
    output1 = chain1.invoke({"topic": text})

    # 두 번째 프롬프트, ChatOpenAI, 문자열 출력 파서를 연결하여 체인을 생성합니다.
    chain2 = prompt2 | ChatOpenAI() | StrOutputParser()
    # 두 번째 체인을 호출하여 파싱된 첫 번째 결과를 전달하고 최종 결과를 반환합니다.
    return chain2.invoke({"sentence": output1})

custom_chain은 이제 실행 가능한 객체(`runnable`)이므로, `invoke()` 를 사용하여 실행해야 합니다.

- LangSmith 추적 기록을 확인해 보면, `custom_chain` 추적 정보가 있을 것이며, 그 아래에는 OpenAI 호출이 중첩되어 있을 것입니다.
- [LangSmith 추적 링크](https://smith.langchain.com/public/17023c06-e53a-4711-a42a-2fbc51249883/r)


In [18]:
# custom_chain을 호출
print(custom_chain.invoke("양자역학"))

🔮✨🌌 양자역학의 신비로운 세계에 온 여러분! 🌌✨🔮 입자들의 확률적인 움직임과 파동으로서의 특성에 대한 물리학적 이론을 탐험해보세요! 🌟🌀💫 양자역학은 현대 물리학의 근간을 이루는 중요한 이론입니다. 💡🔬🌠 함께 양자 세계를 탐험해보세요! 🚀🔭🌟 #양자역학 #물리학 #파동입자이론 #신비로운세계 #현대과학 #인스타그램
